In [1]:
# block
import torch
from torch import nn
from torch.nn import functional as F

"""
In fact, the nn.Sequential() defines a type of special Module, which represents a block in PyTorch.
In addition, we use net(X) to invoke our model and receive the output of it. It is the abbr. of net.__call__(X). This forward propagation simply means that it connects every blocks in the list, and use the output of the previous block as the next block
"""
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)

net(X)

tensor([[ 0.2765,  0.1606, -0.1748,  0.2629, -0.0643, -0.1044,  0.1133,  0.3013,
         -0.0644, -0.0562],
        [ 0.1345,  0.1428, -0.1031,  0.1285,  0.0788,  0.0250,  0.1236,  0.2887,
          0.0818,  0.0536]], grad_fn=<AddmmBackward0>)

In [2]:
# construct a class as block
class MLP(nn.Module):
    # use the parameters of the model to announce layers
    def __init__(self):
        # invoke the super class of the MLP to process necessary initializations
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    # forward propagation
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
"""
Firstly, model will invoke the construction function of its superclass, by __init__() we designed in the model. This operation will decline our efforts on writing code of framework over and over again
Then we implement two FC layers, which is hidden and out. System will help us with backpropagation and parameters initialization
"""
net = MLP()
net(X)

tensor([[ 0.1390,  0.0364, -0.1013,  0.0497, -0.1313, -0.0563, -0.1442,  0.0364,
         -0.0916, -0.2602],
        [-0.0373,  0.0113, -0.0031, -0.0342, -0.0101, -0.0164, -0.2264, -0.0075,
          0.0316, -0.1292]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            #module is an instance of theModule, we save it in the variable _modules, which type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict makes the traversing in the order of adding them
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
"""
__init__() will add each module into the orderedDict _modules one by one orderly, and when we use the forward(), each block we added will execute in the order we add them
"""
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0972, -0.0164, -0.0068,  0.2696, -0.0763,  0.1803,  0.1163,  0.0224,
         -0.3395, -0.1903],
        [-0.0380,  0.1091, -0.0186,  0.1858, -0.1772,  0.1148,  0.1247,  0.0059,
         -0.3682, -0.0590]], grad_fn=<AddmmBackward0>)

In [6]:
# execute control flow in the forward propagation
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # we don't compute the random weight parameters of gradient, so it remains stable during the training process
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)  # this two FC linear shares parameters
        while X.abs().sum() > 1:   # make the L1 norm of the model less than 1, and return the sum of the whole X
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.4580, grad_fn=<SumBackward0>)

In [10]:
# mixture and combination
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0491, grad_fn=<SumBackward0>)